### Step 1: Fetch Historical Data

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from statsmodels.tsa.stattools import adfuller

# Define the tickers for USD/INR and Sensex
usd_inr_ticker = 'INR=X'
sensex_ticker = '^BSESN'

# Define the date range
start_date = '2004-01-01'
end_date = '2015-12-31'

# Fetch the data
usd_inr_data = yf.download(usd_inr_ticker, start=start_date, end=end_date)
sensex_data = yf.download(sensex_ticker, start=start_date, end=end_date)

# Display the first few rows of the data
print(usd_inr_data.tail())
print(sensex_data.tail())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price                      Adj Close      Close       High        Low  \
Ticker                         INR=X      INR=X      INR=X      INR=X   
Date                                                                    
2015-12-24 00:00:00+00:00  66.153999  66.153999  66.094002  65.660004   
2015-12-25 00:00:00+00:00  65.940002  65.940002  66.070000  65.940002   
2015-12-28 00:00:00+00:00  65.940002  65.940002  66.208000  65.940002   
2015-12-29 00:00:00+00:00  66.134003  66.134003  66.408997  66.099998   
2015-12-30 00:00:00+00:00  66.260002  66.260002  66.464996  66.139999   

Price                           Open Volume  
Ticker                         INR=X  INR=X  
Date                                         
2015-12-24 00:00:00+00:00  66.094002      0  
2015-12-25 00:00:00+00:00  65.940002      0  
2015-12-28 00:00:00+00:00  65.959999      0  
2015-12-29 00:00:00+00:00  66.141998      0  
2015-12-30 00:00:00+00:00  66.265999      0  
Price                         Adj Close        

In [8]:
# Calculate log returns
usd_inr_returns = np.log(usd_inr_data['Close'] / usd_inr_data['Close'].shift(1)).dropna()
sensex_returns = np.log(sensex_data['Close'] / sensex_data['Close'].shift(1)).dropna()

# Combine the returns into a single DataFrame using pd.concat to align indices
returns = pd.concat([usd_inr_returns, sensex_returns], axis=1)
returns.columns = ['USD_INR', 'Sensex']
returns = returns.dropna()

# Display the first few rows of the returns
print(returns.head())

                            USD_INR    Sensex
Date                                         
2004-01-05 00:00:00+00:00 -0.003009  0.002057
2004-01-06 00:00:00+00:00  0.001824 -0.015917
2004-01-07 00:00:00+00:00 -0.003563  0.002249
2004-01-08 00:00:00+00:00  0.001783  0.025117
2004-01-09 00:00:00+00:00 -0.002576  0.001807


In [9]:
# Descriptive statistics
descriptive_stats = returns.describe()

# Display the descriptive statistics
print(descriptive_stats)

           USD_INR       Sensex
count  2923.000000  2923.000000
mean      0.000187     0.000511
std       0.005485     0.015435
min      -0.060972    -0.118092
25%      -0.002201    -0.006570
50%       0.000000     0.000951
75%       0.002421     0.008297
max       0.060972     0.159900


In [10]:
# Perform Augmented Dickey-Fuller test
def adf_test(series, title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    print(out.to_string())
    print('')

# Perform ADF test on USD/INR and Sensex returns
adf_test(returns['USD_INR'], title='USD/INR Returns')
adf_test(returns['Sensex'], title='Sensex Returns')

Augmented Dickey-Fuller Test: USD/INR Returns
ADF Test Statistic     -1.531604e+01
p-value                 4.124464e-28
# Lags Used             9.000000e+00
# Observations Used     2.913000e+03
Critical Value (1%)    -3.432597e+00
Critical Value (5%)    -2.862533e+00
Critical Value (10%)   -2.567298e+00

Augmented Dickey-Fuller Test: Sensex Returns
ADF Test Statistic       -39.459291
p-value                    0.000000
# Lags Used                1.000000
# Observations Used     2921.000000
Critical Value (1%)       -3.432591
Critical Value (5%)       -2.862530
Critical Value (10%)      -2.567297

